In [32]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import pandas as pd
from tqdm import tqdm

In [2]:
# !pip install -U torch

In [3]:
!ls ../models/

debert_vk  gemma2-9b-it  vikhr


In [4]:
# ! pip install accelerate>=0.26.0

In [5]:
tokenizer = AutoTokenizer.from_pretrained("../models/gemma2-9b-it")
model = AutoModelForCausalLM.from_pretrained(
    "../models/gemma2-9b-it",
    torch_dtype=torch.bfloat16,
    device_map="cuda:0"
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [57]:
def get_prompt(text):
    
    
    prompt = f"""
        Ты получишь текст, который нужно перефразировать. 
        Твоя задача — сохранить основной смысл текста, но изменить формулировки, включая числа, названия, брендвы и даты. 
        Используй синонимы и близкие по смыслу выражения, сохраняя длину текста приблизительно такой же.

        Теперь обработай следующий текст: {text}
    """

    
    return prompt

    


def get_llm_response(text):
    
    prompt = get_prompt(text)
    
    
    messages = [
        {"role": "user", "content": prompt},
    ]
    input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt", return_dict=True, add_generation_prompt=True).to("cuda")

    outputs = model.generate(**input_ids, 
                             max_new_tokens=100, 
                             do_sample=True,
                             temperature=0.9)
    answer = tokenizer.decode(outputs[0])
    beginning, end = answer.find('<start_of_turn>model\n') + 21, answer.find('\n<end_of_turn') - 1,
    
    return answer[beginning: end]

# Read data

In [58]:
data = pd.read_csv('../data/payments_training.tsv', sep='\t', header=None)
data.rename({
    0: 'id',
    1: 'date',
    2: 'sum',
    3: 'purpose',
    4: 'category'
}, axis=1, inplace=True)

In [59]:
data.head()

,id,date,sum,purpose,category
0,1,07.11.2024,15300.00,За участие в конференции в г. Майкоп по догово...,SERVICE
1,2,07.11.2024,"40200,00",За оказание услуг по договору №79-02726В от 01...,SERVICE
2,3,07.11.2024,1440-00,Оплата за Порошок стиральный Ariel Color autom...,NON_FOOD_GOODS
3,4,07.11.2024,240000000-00,Возврат денежных средств по договору займа №04...,LOAN
4,5,07.11.2024,1360000.00,"Оплата Дог №452 от 13/03/2021, согл. Сч 0745-2...",NOT_CLASSIFIED


In [60]:
data['category'].value_counts()

NON_FOOD_GOODS    96
FOOD_GOODS        90
SERVICE           88
BANK_SERVICE      49
TAX               48
LOAN              41
LEASING           38
REALE_STATE       27
NOT_CLASSIFIED    23
Name: category, dtype: int64

In [61]:
data[data['category'] == 'NOT_CLASSIFIED']['purpose'].iloc[12]

'Перевод себе'

In [62]:
n = 423


text = data.iloc[n]['purpose']
label = data.iloc[n]['category']
text, label

('государственная пошлина', 'TAX')

In [63]:
response = get_llm_response(text)
response

'Налог, взимаемый государством '

In [64]:
data['purpose'].nunique()

406

In [65]:
data

,id,date,sum,purpose,category
0,1,07.11.2024,15300.00,За участие в конференции в г. Майкоп по догово...,SERVICE
1,2,07.11.2024,"40200,00",За оказание услуг по договору №79-02726В от 01...,SERVICE
2,3,07.11.2024,1440-00,Оплата за Порошок стиральный Ariel Color autom...,NON_FOOD_GOODS
3,4,07.11.2024,240000000-00,Возврат денежных средств по договору займа №04...,LOAN
4,5,07.11.2024,1360000.00,"Оплата Дог №452 от 13/03/2021, согл. Сч 0745-2...",NOT_CLASSIFIED
...,...,...,...,...,...
495,496,07.11.2024,2610.00,Оплата налогов,TAX
496,497,07.11.2024,31200-00,Комиссия за выполнение функций агента валютног...,BANK_SERVICE
497,498,07.11.2024,18200-00,За тур.поездку по договору №75-04243Г от 24/04...,SERVICE
498,499,07.11.2024,"2870000,00","Оплата по договору №095 от 24.02.2025г, счету ...",FOOD_GOODS


In [66]:
ids = data['id'].values
purpose = data['purpose'].values
category = data['category'].values

augmented_texts = []


for text_id, purpose, category in tqdm(zip(ids, purpose, category)):
    augmented_text = get_llm_response(purpose)
    augmented_texts.append(augmented_text)


500it [39:04,  4.69s/it]


In [ ]:
data = pd.read_csv('../data/augmented.csv')

In [55]:
data['augmented_texts2'] = augmented_texts

In [56]:
data.to_csv('../data/augmented.csv')